## Camera Calibration

Relevant information can be found in the OpenCV [OpenCV](https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html) documentation

In [15]:
# First create a python virtual environment

# Install this packages
#!pip install numpy
#!pip install opencv-python

Obtain objpoints from the images

In [2]:
import numpy as np
import cv2 as cv
import glob
import os

chessboardSize = (9,6)

# termination criteria for refining corner points
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0), ....,(6,5,0)
objp = np.zeros((chessboardSize[0] * chessboardSize[1], 3), np.float32)
objp[:,:2] = np.mgrid[0:chessboardSize[0],0:chessboardSize[1]].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = []  # 3d points in real world space
imgpoints = []  # 2d points in image plane.

# Get all image file paths from the folder
images = glob.glob('Calibration_Images/Images/*.jpg')

# Create directory for saving results if not exists
output_dir = "Calibration_Images/Corner_Results"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for idx, fname in enumerate(images):
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv.findChessboardCorners(gray, chessboardSize, None)

    # If corners are found
    if ret:
        objpoints.append(objp)

        # Refine corner locations
        corners2 = cv.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        cv.drawChessboardCorners(img, chessboardSize, corners2, ret)

        # Save the result with corners drawn
        result_filename = os.path.join(output_dir, f"corner_result_{idx+1}.jpg")
        cv.imwrite(result_filename, img)

Obtain calibration camera results

In [3]:
# Calibrate the camera
ret, camera_matrix, distortion_coeffs, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# Print calibration results
print("Camera matrix:\n", camera_matrix)
print("Distortion coefficients:\n", distortion_coeffs)
print("Rotation Vectors:\n", rvecs)
print("Translation Vectors:\n", tvecs)

Camera matrix:
 [[954.12026393   0.         650.53788058]
 [  0.         956.50374732 358.19426202]
 [  0.           0.           1.        ]]
Distortion coefficients:
 [[ 0.02196937 -0.18365092  0.00141952 -0.00032658  0.22389322]]
Rotation Vectors:
 (array([[-0.24425083],
       [ 0.66263672],
       [ 0.00457787]]), array([[ 0.0274856 ],
       [-0.00998668],
       [ 0.01979025]]), array([[-0.79319553],
       [-0.31243644],
       [-0.69060952]]), array([[-0.72957594],
       [ 0.5068719 ],
       [-0.05314321]]), array([[ 0.4043267 ],
       [-0.36419681],
       [ 0.56793921]]), array([[0.0474642 ],
       [0.01451282],
       [0.00163657]]), array([[-0.09837335],
       [-0.76452522],
       [ 0.0207407 ]]), array([[ 0.61342418],
       [-0.03131579],
       [ 0.01128204]]), array([[-0.08033699],
       [-0.06413614],
       [ 0.4872341 ]]), array([[0.80455581],
       [0.01169279],
       [0.00921147]]), array([[-0.06156876],
       [-0.0164725 ],
       [-0.00122942]]), array

Undistort an image

In [4]:
# Undistort one of the test images
img = cv.imread(images[5])
h,  w = img.shape[:2]
new_camera_matrix, roi = cv.getOptimalNewCameraMatrix(camera_matrix, distortion_coeffs, (w,h), 1, (w,h))

# Undistort
undistorted_img = cv.undistort(img, camera_matrix, distortion_coeffs, None, new_camera_matrix)

# Crop the image if necessary
x, y, w, h = roi
undistorted_img = undistorted_img[y:y + h, x:x + w]

# Save and display the undistorted image
cv.imwrite('Calibration_Images/undistorted_result.jpg', undistorted_img)

True

### Reprojection Error

In [5]:
def compute_reprojection_error(objpoints, imgpoints, rvecs, tvecs, camera_matrix, distortion_coeffs):
    total_error = 0
    total_points = 0

    # Loop through each set of object points and image points
    for i in range(len(objpoints)):
        # Project 3D object points to 2D image points using the calibrated camera parameters
        imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], camera_matrix, distortion_coeffs)
        
        # Compute the error (difference) between the projected points and the actual 2D image points
        error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2) / len(imgpoints2)
        total_error += error
        total_points += len(imgpoints2)

    # Average reprojection error
    mean_error = total_error / len(objpoints)
    
    return mean_error

# Call this after the calibration
reprojection_error = compute_reprojection_error(objpoints, imgpoints, rvecs, tvecs, camera_matrix, distortion_coeffs)
print(f"Reprojection Error: {reprojection_error}")

Reprojection Error: 0.04953106145322064
